# Roubos de celular na cidade de São Paulo

Nesse notebook, vamos filtrar e tratar os dados de roubos de celulares ocorridos na cidade de São Paulo, utilizando Jupyter e GeoPandas

In [1]:
# Importando a biblioteca Pandas para leitura dos dados
import pandas as pd

In [4]:
# Lendo os dados
filename = 'data/dados_roubo_celular_sp_2024.xlsx'

df_roubos = pd.read_excel(filename)

In [5]:
df_roubos.head()

,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,NOME_DELEGACIA,NOME_MUNICIPIO,ANO_BO,NUM_BO,VERSAO,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,...,DESCR_MODO_OBJETO,DESCR_TIPO_OBJETO,DESCR_SUBTIPO_OBJETO,DESCR_UNIDADE,QUANTIDADE_OBJETO,MARCA_OBJETO,FLAG_BLOQUEIO,FLAG_DESBLOQUEIO,MES,ANO
0,10109,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,2024,AL6931,1,DECAP,DEL.SEC.1º CENTRO,...,Subtraído,Telecomunicação,Telefone Celular,Unidade,1,Samsung,N,N,1,2024
1,10109,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,2024,AL6931,1,DECAP,DEL.SEC.1º CENTRO,...,Subtraído,Telecomunicação,Telefone Celular,Unidade,1,Samsung,N,N,1,2024
2,10109,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,2024,AN9687,1,DECAP,DEL.SEC.1º CENTRO,...,Subtraído,Telecomunicação,Telefone Celular,Unidade,1,Outros,S,N,1,2024
3,10109,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,2024,AH3601,2,DECAP,DEL.SEC.7º ITAQUERA,...,Subtraído,Telecomunicação,Telefone Celular,Unidade,1,Samsung,N,N,1,2024
4,10109,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,2024,AH3601,2,DECAP,DEL.SEC.7º ITAQUERA,...,Subtraído,Telecomunicação,Telefone Celular,Unidade,1,Samsung,N,N,1,2024


In [6]:
# Mostrar todas as colunas para verificar se temos informações de latitude e longitude
df_roubos.columns

Index(['ID_DELEGACIA', 'NOME_DEPARTAMENTO', 'NOME_SECCIONAL', 'NOME_DELEGACIA',
       'NOME_MUNICIPIO', 'ANO_BO', 'NUM_BO', 'VERSAO',
       'NOME_DEPARTAMENTO_CIRC', 'NOME_SECCIONAL_CIRC', 'NOME_DELEGACIA_CIRC',
       'NOME_MUNICIPIO_CIRC', 'DATA_OCORRENCIA_BO', 'HORA_OCORRENCIA',
       'DESCRICAO_APRESENTACAO', 'DATAHORA_REGISTRO_BO', 'DATA_COMUNICACAO_BO',
       'DATAHORA_IMPRESSAO_BO', 'DESCR_PERIODO', 'AUTORIA_BO',
       'FLAG_INTOLERANCIA', 'TIPO_INTOLERANCIA', 'FLAG_FLAGRANTE',
       'FLAG_STATUS', 'DESC_LEI', 'FLAG_ATO_INFRACIONAL', 'RUBRICA',
       'DESCR_CONDUTA', 'DESDOBRAMENTO', 'CIRCUNSTANCIA', 'DESCR_TIPOLOCAL',
       'DESCR_SUBTIPOLOCAL', 'CIDADE', 'BAIRRO', 'CEP', 'LOGRADOURO_VERSAO',
       'LOGRADOURO', 'NUMERO_LOGRADOURO', 'LATITUDE', 'LONGITUDE',
       'CONT_OBJETO', 'DESCR_MODO_OBJETO', 'DESCR_TIPO_OBJETO',
       'DESCR_SUBTIPO_OBJETO', 'DESCR_UNIDADE', 'QUANTIDADE_OBJETO',
       'MARCA_OBJETO', 'FLAG_BLOQUEIO', 'FLAG_DESBLOQUEIO', 'MES', 'ANO'],
      d

In [7]:
# Mostrar apenas os dados de latitude e longitude
df_roubos[['LATITUDE', 'LONGITUDE']]

,LATITUDE,LONGITUDE
0,0.000000,0.000000
1,0.000000,0.000000
2,-23.528590,-46.606265
3,0.000000,0.000000
4,0.000000,0.000000
...,...,...
111022,-23.488858,-46.375312
111023,-23.488858,-46.375312
111024,-23.488858,-46.375312
111025,-23.488858,-46.375312


In [9]:
# Removendo valores zerados ou NaN
df_roubos = df_roubos.dropna(subset=['LATITUDE', 'LONGITUDE'])
df_roubos = df_roubos[(df_roubos['LATITUDE'] != 0) | (df_roubos['LONGITUDE'] != 0)]
df_roubos[['LATITUDE', 'LONGITUDE']]

,LATITUDE,LONGITUDE
2,-23.528590,-46.606265
9,-23.559632,-46.634949
10,-23.559632,-46.634949
11,-23.552427,-46.629118
12,-23.552063,-46.634225
...,...,...
111022,-23.488858,-46.375312
111023,-23.488858,-46.375312
111024,-23.488858,-46.375312
111025,-23.488858,-46.375312


## Criando o GeoDataframe (GeoPandas)

In [10]:
# Importando a biblioteca Shapely
from shapely.geometry import Point

In [11]:
# Criando a coluna geometry
df_roubos['geometry'] = None

for index, row in df_roubos.iterrows():
    df_roubos.loc[index, 'geometry'] = Point(row.LONGITUDE, row.LATITUDE)

In [12]:
# Mostrando dos dados
df_roubos.head(5)

,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,NOME_DELEGACIA,NOME_MUNICIPIO,ANO_BO,NUM_BO,VERSAO,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,...,DESCR_TIPO_OBJETO,DESCR_SUBTIPO_OBJETO,DESCR_UNIDADE,QUANTIDADE_OBJETO,MARCA_OBJETO,FLAG_BLOQUEIO,FLAG_DESBLOQUEIO,MES,ANO,geometry
2,10109,DECAP,DEL.SEC.1º CENTRO,01ª DDM CENTRO,S.PAULO,2024,AN9687,1,DECAP,DEL.SEC.1º CENTRO,...,Telecomunicação,Telefone Celular,Unidade,1,Outros,S,N,1,2024,POINT (-46.6062647869999 -23.5285897549999)
9,10101,DECAP,DEL.SEC.1º CENTRO,01º D.P. SE,S.PAULO,2024,AB5526,2,DECAP,DEL.SEC.1º CENTRO,...,Telecomunicação,Telefone Celular,Unidade,1,Samsung,NaN,NaN,1,2024,POINT (-46.6349487 -23.5596324)
10,10101,DECAP,DEL.SEC.1º CENTRO,01º D.P. SE,S.PAULO,2024,AB5526,2,DECAP,DEL.SEC.1º CENTRO,...,Telecomunicação,Telefone Celular,Unidade,1,LG,NaN,NaN,1,2024,POINT (-46.6349487 -23.5596324)
11,10101,DECAP,DEL.SEC.1º CENTRO,01º D.P. SE,S.PAULO,2024,AB6330,1,DECAP,DEL.SEC.1º CENTRO,...,Telecomunicação,Telefone Celular,Unidade,1,Samsung,S,N,1,2024,POINT (-46.6291179 -23.5524268)
12,10101,DECAP,DEL.SEC.1º CENTRO,01º D.P. SE,S.PAULO,2024,AB6659,1,DECAP,DEL.SEC.1º CENTRO,...,Telecomunicação,Telefone Celular,Unidade,1,Motorola,S,N,1,2024,POINT (-46.6342252 -23.5520634)


In [13]:
# Verificando o tipo do dataframe
type(df_roubos)

pandas.core.frame.DataFrame

In [14]:
# Importando GeoPandas
import geopandas as gpd

In [16]:
# Criando o GeoDataFrame
gdf_roubos = gpd.GeoDataFrame(df_roubos, geometry='geometry')

In [17]:
# Verificando tipo
type(gdf_roubos)

geopandas.geodataframe.GeoDataFrame